In [2]:
import bibtexparser

with open('works.bib') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

In [18]:
bib_database.entries[10]

{'year': '2018',
 'url': 'http://doi.org/10.1117/12.2314090',
 'doi': '10.1117/12.2314090',
 'author': "Lionel Garcia and Garcia, Lionel and Prod{'}homme, Thibaut and Lucsanyi, David and Short, Alex and Santin, Giovanni and Vuolo, Marco and Crowley, Cian and Abreu, Asier and Kholey, Ralf and Lemmel, Frederic and Smit, Hans and de Bruijne, Jos",
 'journal': 'High Energy, Optical, and Infrared Detectors for Astronomy VIII',
 'title': 'Validation of a CCD cosmic ray event simulator against Gaia in-orbit data',
 'ENTRYTYPE': 'conference',
 'ID': 'Lionel_Garcia_91723392'}

In [53]:
txts = []

def rem_braces(t):
    return t.replace("{", "").replace("}", "").replace("\pi", "$\pi$")

for paper in bib_database.entries:
    authors = rem_braces(paper["author"])
    authors = ", ".join(authors.split(" and ")[1:4]) + " et al."
    if "Garcia" not in authors:
        authors += " (including Garcia, L. J.)"
    else:
        authors.replace("Garcia, Lionel", "\\textbf{Garcia, Lionel}")
        
    title = rem_braces(paper["title"])
    date = rem_braces(paper["year"])
    if "publisher" in paper:
        journal = rem_braces(paper["publisher"])
    else:
        journal = ""
    txts.append(f"""\\publi{{{date}}}
    {{{title}}}
    {{{authors}}}
    {{{journal}}}""")
    
open("works.txt", "w").write("\n\n".join(txts))

2089

In [43]:
txts

['\\publi{2021}\n    {SPECULOOS: Ultracool dwarf transit survey}\n    {D. Sebastian,M. Gillon,E. Ducrot et al.(including Garcia, L. J.)}\n    {{EDP} Sciences}',
 '\\publi{2021}\n    {Six transiting planets and a chain of Laplace resonances in TOI-178}\n    {Lionel Garcia,Leleu, A.,Alibert, Y. et al.}\n    {}',
 '\\publi{2020}\n    {Development of the SPECULOOS exoplanet search project}\n    {Lionel Garcia,Sebastian, D.,Pedersen, P. P. et al.}\n    {}',
 '\\publi{2020}\n    {VizieR Online Data Catalog: SPECULOOS. Ultracool dwarf transit survey (Sebastian+, 2021)}\n    {Lionel Garcia,Sebastian, D.,Gillon, M. et al.}\n    {}',
 '\\publi{2020}\n    {A super-Earth and a sub-Neptune orbiting the bright, quiet M3 dwarf TOI-1266}\n    {Lionel Garcia,Demory, B. -O.,Pozuelos, F. J. et al.}\n    {}',
 "\\publi{2020}\n    {\\pi Earth: A 3.14 day Earth-sized Planet from K2's Kitchen Served Warm by the SPECULOOS Team}\n    {Lionel Garcia,Niraula, Prajwal,Wit, Julien de et al.}\n    {}",
 "\\publi{20

In [18]:
paper

{'journal': 'Astronomy {\\&} Astrophysics',
 'title': '{SPECULOOS}: Ultracool dwarf transit survey',
 'author': 'D. Sebastian and M. Gillon and E. Ducrot and F. J. Pozuelos and L. J. Garcia and M. N. GÃ¼nther and L. Delrez and D. Queloz and B. O. Demory and A. H. M. J. Triaud and A. Burgasser and J. de Wit and A. Burdanov and G. Dransfield and E. Jehin and J. McCormac and C. A. Murray and P. Niraula and P. P. Pedersen and B. V. Rackham and S. Sohy and S. Thompson and V. Van Grootel',
 'pages': 'A100',
 'volume': '645',
 'publisher': '{EDP} Sciences',
 'month': 'jan',
 'year': '2021',
 'url': 'https://doi.org/10.1051%2F0004-6361%2F202038827',
 'doi': '10.1051/0004-6361/202038827',
 'ENTRYTYPE': 'article',
 'ID': 'Sebastian_2021'}

In [73]:
ads.config.token = '4Wb11yxAAyjXjffgcGsrLmopqA6rdagUwuNtDhTA'
papers = list(
    ads.SearchQuery(
        author="Garcia, Lionel J.",
        fl=[
            "id",
            "title",
            "author",
            "doi",
            "year",
            "pubdate",
            "pub",
            "volume",
            "page",
            "identifier",
            "doctype",
            "citation_count",
            "bibcode",
            "citation",
        ],
        max_pages=200,
    )
)

In [74]:
for paper in papers:
    if "planet" in paper.title:
        print(paper.title)

In [2]:
def get_papers(author, count_cites=True):
    papers = list(
        ads.SearchQuery(
            author=author,
            fl=[
                "id",
                "title",
                "author",
                "doi",
                "year",
                "pubdate",
                "pub",
                "volume",
                "page",
                "identifier",
                "doctype",
                "citation_count",
                "bibcode",
                "citation",
            ],
            max_pages=100,
        )
    )
    dicts = []

    # Count the citations as a function of time
    citedates = []

    # Save bibcodes for later
    bibcodes = []

    for paper in papers:

        if not (
            ("Luger, Rodrigo" in paper.author)
            or ("Luger, R." in paper.author)
            or ("Luger, R" in paper.author)
        ):
            continue

        if manual_exclude(paper):
            continue

        aid = [
            ":".join(t.split(":")[1:])
            for t in paper.identifier
            if t.startswith("arXiv:")
        ]
        for t in paper.identifier:
            if len(t.split(".")) != 2:
                continue
            try:
                list(map(int, t.split(".")))
            except ValueError:
                pass
            else:
                aid.append(t)
        try:
            page = int(paper.page[0])
        except ValueError:
            page = None
            if paper.page[0].startswith("arXiv:"):
                aid.append(":".join(paper.page[0].split(":")[1:]))
        except TypeError:
            page = None

        # Get citation dates
        if count_cites and paper.citation is not None:
            for i, bibcode in tqdm(
                enumerate(paper.citation), total=len(paper.citation)
            ):
                try:
                    cite = list(ads.SearchQuery(bibcode=bibcode, fl=["pubdate"]))[0]
                    date = int(cite.pubdate[:4]) + int(cite.pubdate[5:7]) / 12.0
                    citedates.append(date)
                except IndexError:
                    pass

        # Save bibcode
        bibcodes.append(paper.bibcode)

        dicts.append(
            dict(
                doctype=paper.doctype,
                authors=format_authors(paper.author),
                year=paper.year,
                pubdate=paper.pubdate,
                doi=paper.doi[0] if paper.doi is not None else None,
                title=format_title(paper.title[0]),
                pub=paper.pub,
                volume=paper.volume,
                page=page,
                arxiv=aid[0] if len(aid) else None,
                citations=paper.citation_count,
                url="http://adsabs.harvard.edu/abs/" + paper.bibcode,
            )
        )

    if count_cites:
        # Sort the cite dates
        citedates = sorted(citedates)
        np.savetxt("citedates.txt", citedates, fmt="%.3f")

    # Save bibcodes
    with open("bibcodes.txt", "w") as f:
        for bibcode in bibcodes:
            print(bibcode, file=f)

    return sorted(dicts, key=itemgetter("pubdate"), reverse=True)


In [39]:
for r in s.get():
    print(r.title, r.authors)

Direct laser writing for cardiac tissue engineering: a microfluidic heart on a chip with integrated transducers [<arxiv.arxiv.Result.Author object at 0x108f85e10>, <arxiv.arxiv.Result.Author object at 0x108fab710>, <arxiv.arxiv.Result.Author object at 0x108fab350>, <arxiv.arxiv.Result.Author object at 0x108fab390>, <arxiv.arxiv.Result.Author object at 0x108fab450>, <arxiv.arxiv.Result.Author object at 0x108f9ca50>, <arxiv.arxiv.Result.Author object at 0x108faac90>, <arxiv.arxiv.Result.Author object at 0x108f96d50>, <arxiv.arxiv.Result.Author object at 0x108fbd190>]
The XFaster Power Spectrum and Likelihood Estimator for the Analysis of Cosmic Microwave Background Maps [<arxiv.arxiv.Result.Author object at 0x108ede710>, <arxiv.arxiv.Result.Author object at 0x108f85e50>, <arxiv.arxiv.Result.Author object at 0x108fa2c50>, <arxiv.arxiv.Result.Author object at 0x108fa2150>, <arxiv.arxiv.Result.Author object at 0x108fa2390>, <arxiv.arxiv.Result.Author object at 0x108fa2a50>, <arxiv.arxiv.Res

In [18]:
list(res)

[]

In [34]:
import urllib
url = 'http://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=1'
data = urllib.urlopen(url).read()
print(data)

AttributeError: module 'urllib' has no attribute 'urlopen'